In [18]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

%matplotlib inline

## Model the matches

In [19]:
yr_ls = [2007 + i for i in range(9)]
nfl_dict = {}
for yr in year_ls:
    nfl_dict[yr] = pd.read_csv("NFL_data/"+str(yr)+".csv")
    nfl_dict[yr].columns = ['win', 'loser', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL',
       'TOL']

In [25]:
ls1 = [i for i in nfl_dict[2007]["win"]]
ls2 = [i for i in nfl_dict[2007]["loser"]]

teams = list(set(ls1 + ls2))
teams.sort()

# Create y and n matrix for each year

In [23]:
ymat_dict = {}
nmat_dict = {}
for yr in yr_ls:
    ymat_dict[yr] = pd.DataFrame(np.zeros((32,32)), columns=teams, index=teams)
    nmat_dict[yr] = pd.DataFrame(np.zeros((32,32)), columns=teams, index=teams)
    for row in range(nfl_dict[yr].shape[0]):
        i = nfl_dict[yr].loc[row, "win"]
        j = nfl_dict[yr].loc[row, "loser"]
        ymat_dict[yr].loc[i, j] += 1
    nmat_dict[yr] = ymat_dict[yr] + ymat_dict[yr].T
    nmat_dict[yr] = pd.DataFrame(np.tril(nmat_dict[yr]), index=teams, columns=teams)
    ymat_dict[yr] = pd.DataFrame(np.tril(ymat_dict[yr]), index=teams, columns=teams)

In [24]:
ymat_ls = []
nmat_ls = []
for yr in yr_ls:
    ymat_ls.append(ymat_dict[yr])
    nmat_ls.append(nmat_dict[yr])

# Create a feature matrix

In [36]:
# Points won
nfl_dict[2008].groupby("loser").mean()

,PtsW,PtsL,YdsW,TOW,YdsL,TOL
loser,,,,,,
Arizona Cardinals,37.625000,21.000000,375.875000,0.625000,343.250000,2.875000
Atlanta Falcons,26.333333,16.833333,374.500000,0.500000,310.833333,1.666667
Baltimore Ravens,22.166667,11.000000,286.666667,1.166667,243.833333,2.666667
Buffalo Bills,23.444444,13.333333,315.333333,0.666667,279.444444,2.777778
Carolina Panthers,31.800000,16.400000,366.200000,0.600000,301.200000,2.200000
Chicago Bears,27.428571,16.571429,372.857143,1.428571,288.714286,1.571429
Cincinnati Bengals,27.166667,11.083333,353.666667,1.333333,227.416667,1.916667
Cleveland Browns,23.333333,10.416667,367.166667,1.500000,217.166667,2.000000
Dallas Cowboys,31.714286,17.000000,318.571429,1.428571,308.000000,3.142857


In [37]:
def feature_gen(data):
    hold = data.groupby("win").mean()
    hold["avg_PtsL"] = data.groupby('loser').mean()["PtsL"]
    hold["avg_YdsL"] = data.groupby('loser').mean()["YdsL"]
    return hold[["PtsW", "avg_PtsL","YdsW","avg_YdsL"]]
    

In [38]:
feature_ls = []
for yr in yr_ls:
    feature_ls.append(feature_gen)

,PtsW,avg_PtsL,YdsW,avg_YdsL
win,,,,
Arizona Cardinals,31.125000,19.375000,355.250000,332.875000
Atlanta Falcons,27.500000,12.416667,307.750000,298.500000
Baltimore Ravens,20.800000,15.545455,316.200000,295.545455
Buffalo Bills,21.428571,11.333333,333.142857,233.555556
Carolina Panthers,24.285714,10.777778,329.857143,250.000000
Chicago Bears,26.857143,16.222222,287.571429,297.666667
Cincinnati Bengals,28.142857,20.333333,345.142857,350.222222
Cleveland Browns,29.100000,18.500000,381.300000,301.333333
Dallas Cowboys,32.000000,14.000000,398.384615,252.000000
